# Feature Extraction from Aerial Images (Didn't Work!)
In this repository, we extract features from a set of satellite images using the pre-trained models in [ssai-cnn](https://github.com/mitmul/ssai-cnn/wiki/Pre-trained-models) repository.
We here use two models **MnihCNN_cis** and **MnihCNN_multi**, which are implementations of the methods in [Volodymyr Mnih's PhD thesis](https://www.cs.toronto.edu/~vmnih/docs/Mnih_Volodymyr_PhD_Thesis.pdf).

The code below depends on the libraries below:

- Python 3.5 (anaconda with python 3.5.1 is recommended)
  - Chainer 1.5.0.2
  - Cython 0.23.4
  - NumPy 1.10.1
  - tqdm
- OpenCV 3.0.0

### Load the model

In [ ]:
import chainer
from mnih_models import MnihCNN_multi
from config import MODEL_PATH

model = MnihCNN_multi()
chainer.serializers.load_hdf5(MODEL_PATH,model)
model.train = False

### Extract the features

In [2]:
import glob
from scipy import misc
import cv2 as cv
from chainer import Variable
import numpy as np
import predict
import os
from chainer import cuda
from config import IMG_PATH
args = predict.get_args()


image_paths = glob.glob(IMG_PATH+'*.jpg')[:1]
preds = []
for i,path in enumerate(image_paths):
    if not (i+1)%1000:
        print(i+1,'/',len(image_paths))
        
    img = cv.resize(cv.imread(path),(args.sat_size,args.sat_size))
    x=model(np.array([img.transpose((2, 0, 1))],dtype=np.float32),None,last_layer=True)
    # OR
#   x=get_predict(args,img,model)
    preds.append(x.data.flatten())

### Save the features

In [6]:
import os
import pickle
img_idx = {os.path.basename(path):i for i,path in enumerate(image_paths)}

with open('features.pkl','bw') as f:
    pickle.dump({
       'img_idx' : img_idx,
        'preds'  : preds
    },f)